## Previsão de Inadimplência

### Leitura dos Dados

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_parquet('data/train.parquet')
test_df = pd.read_parquet('data/test.parquet')
sample = pd.read_parquet('data/sub.parquet')

### Análise Exploratória

Inicialmente observamos que a base de dados de empréstimos contém dados com granularidade diária sobre o empréstimo e o estado atual do empréstimo. O empréstimo possuí variáveis constantes, como
o dia de desembolso do valor, a data de vencimento, o valor emprestado e o pagamento diário esperado. Já seu estado diário inclui o valor da dívida total atualizada, o pagamento do dia e o valor total das transações do dia. Cada empréstimo está associado a um identificador único e a um segmento comercial.

O objetivo do case é identificar quais empréstimos tem maior chance de inadimplência, em particular vamos **identificar quais empréstimos não serão pagos com dados dos primeiros 90 dias.** Logo, nos limitamos a usar apenas os dados dos primeiros 90 dias, apesar do conjunto de dados apresentar dados de empréstimos com durações maiores.

In [3]:
train_df

,id,desembolso,vencimento,valor_emprestado,pgto_diario_esperado,dia,dias_pos_desembolso,divida_total,divida_principal,pagamento_diario,amortizacao_principal_diario,transacionado,subsegmento,y
3281,6.0,2019-03-25,2019-12-26,7100.0,33.7107,2019-03-25,0.0,7197.98,7100.0,0.00,0.0,664.00,None,0.0
3282,6.0,2019-03-25,2019-12-26,7100.0,33.8852,2019-03-26,1.0,7212.25,7100.0,0.00,0.0,0.00,None,0.0
3283,6.0,2019-03-25,2019-12-26,7100.0,33.8852,2019-03-27,2.0,7226.54,7100.0,0.00,0.0,150.00,None,0.0
3284,6.0,2019-03-25,2019-12-26,7100.0,33.8852,2019-03-28,3.0,7207.22,7100.0,33.64,0.0,0.00,None,0.0
3285,6.0,2019-03-25,2019-12-26,7100.0,33.8852,2019-03-29,4.0,7221.51,7100.0,0.00,0.0,0.00,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29330789,59438.0,2021-02-23,2022-05-21,8769.0,29.5007,2021-02-28,5.0,8844.83,8769.0,0.00,0.0,0.00,None,0.0
29337013,59443.0,2021-02-25,2021-08-23,8902.0,56.0107,2021-02-25,0.0,8935.83,8902.0,0.00,0.0,16.70,None,0.0
29337015,59443.0,2021-02-25,2021-08-23,8902.0,56.2093,2021-02-26,1.0,8945.30,8902.0,2.56,0.0,183.43,None,0.0
29337017,59443.0,2021-02-25,2021-08-23,8902.0,56.2093,2021-02-27,2.0,8957.34,8902.0,0.00,0.0,0.00,None,0.0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2889113 entries, 3281 to 29337019
Data columns (total 14 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   id                            float64       
 1   desembolso                    datetime64[ns]
 2   vencimento                    datetime64[ns]
 3   valor_emprestado              float64       
 4   pgto_diario_esperado          float64       
 5   dia                           datetime64[ns]
 6   dias_pos_desembolso           float64       
 7   divida_total                  float64       
 8   divida_principal              float64       
 9   pagamento_diario              float64       
 10  amortizacao_principal_diario  float64       
 11  transacionado                 float64       
 12  subsegmento                   object        
 13  y                             float64       
dtypes: datetime64[ns](3), float64(10), object(1)
memory usage: 330.6+ MB


---
Verificamos que existe desbalanceamento nos dados, isto é, apenas cerca de 4.5% são empréstimos não são pagos. Este é um ponto importante que influencia como o modelo será avaliado. Desta forma vamos otimizar os modelos para recuperar o maior número possível de defaults, em detrimento da acurácia da classe.

In [5]:
targets_freq = train_df.groupby('id').first()
targets_freq = targets_freq.y.value_counts()
print(targets_freq)

0.0    16576
1.0      768
Name: y, dtype: int64


---
Em relação a saúde dos dados, podemos verificar se a predição do cliente quitar a dívida se relaciona com o dia do empréstimo ou não. Para este fim calculamos o mínimo e máximo da variável de classe de um mesmo identificador. E como os valores foram semelhantes, podemos dizer que cada identificador equivale a apenas uma classe. Isto é, a predição de um dia de empréstimo tem relação apenas com o estado de quitar a dívida.

In [6]:
for i in [0, 1]:
    ids = train_df.query(f'y == {i}')[['id', 'y']]
    neg_ids = train_df.groupby('id').min()
    pos_ids = train_df.groupby('id').max()
    mismatch_n = (neg_ids != pos_ids).sum().values[0]
    print(f'Número de incompatibilidades na classe {i}: {mismatch_n}')

Número de incompatibilidades na classe 0: 0
Número de incompatibilidades na classe 1: 0


---
Outra questão importante é descobrir se algumas industrias tem maior probabilidade de default. Abordamos essa questão verificando a probabilidade de os empréstimos de um segmento apresentar default, calculando a quantidade de empréstimos que apresentaram default dividida pela quantidade total de empréstimos do segmento. No entanto o resultado foi inconclusivo, pois o tamanho do conjunto de treinamento é tal que muitos segmentos tem poucas amostras. Notamos que 57 segmentos diferentes têm menos de 100 amostras. Por este motivo não iremos incorporar esta feature a modelagem.

In [7]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

tot_segment = train_df.groupby('id').first().subsegmento.value_counts()
pos_segment = train_df.query('y == 1').groupby('id').first().subsegmento.value_counts()
neg_segment = train_df.query('y == 0').groupby('id').first().subsegmento.value_counts()

rsk_segment = pos_segment.divide(tot_segment, fill_value=0).sort_values(ascending=False)

In [8]:
print('Probabilidade "a priori" de um segmento obter default:\n')
print_full(rsk_segment)

Probabilidade "a priori" de um segmento obter default:

Ensino Superior e Técnico                     1.000000
Entretenimento e Turismo                      0.217391
Táxi/Carona                                   0.200000
Joalherias, Relojoarias e Pratarias           0.166667
Lojas de Departamento                         0.166667
Consultorias                                  0.147059
Estacionamentos e Lava-rápidos                0.145161
Serviços Imobiliários                         0.142857
Gráfica, Impressão e Xerox                    0.142857
Armarinhos e Tecido                           0.139535
Festas e Eventos                              0.137931
Óticas e Óculos                               0.125000
Outros Serviços - Outros                      0.112360
Vestuário                                     0.112230
Oficinas Automotivas                          0.110687
Móveis                                        0.106195
Cama, Mesa e Banho                            0.100000
Outros Pr

In [9]:
print(f'Quantidade de segmentos com menos de 100 amostras: {(tot_segment <= 100).sum()}')
print(f'Quantidade total de segmentos: {tot_segment.size}')

Quantidade de segmentos com menos de 100 amostras: 57
Quantidade total de segmentos: 73


---
É recomendado também analisar as correlações entre as features. Logo verificamos dois grupos de features:
- Divida total, divida principal e transacionado
- Pagamento diário, amortização principal diária e transacionado  

Então identificamos features que não serão utilizadas, seja por serem constantes ou pela alta colinearidade, como:
- desembolso
- vencimento
- valor_emprestado
- pgto_diario_esperado
- dia
- dias_pos_desembolso
- divida_principal
- subsegmento

In [10]:
train_df[['divida_total', 'divida_principal', 'transacionado']].corr()

,divida_total,divida_principal,transacionado
divida_total,1.000000,0.995532,0.256721
divida_principal,0.995532,1.000000,0.270050
transacionado,0.256721,0.270050,1.000000


In [11]:
train_df[['pgto_diario_esperado', 'amortizacao_principal_diario', 'transacionado']].corr()

,pgto_diario_esperado,amortizacao_principal_diario,transacionado
pgto_diario_esperado,1.000000,0.218605,0.340946
amortizacao_principal_diario,0.218605,1.000000,0.150017
transacionado,0.340946,0.150017,1.000000


---
Apesar da alta quantidade de features eliminadas o modelo de classificação não levara em conta a ordem temporal.